In [1]:
import numpy as np
import matplotlib
import pandas as pd
from IPython.display import Image
Image("convTimes.png")

ModuleNotFoundError: No module named 'numpy'

Explanation of below functions:  
***Continuous conversion mode:*** The device continuously performs temperature conversions in this mode, as shown in Figure 7-3, and updates the temperature result register at the end of every active conversion  
***One-shot mode:*** After the device completes a (temperature conversion) one-shot conversion, the device goes to the *low-power shutdown mode*.  

In [ ]:
def ContinuousConversion(CC_params=[averaging, convCycleTime]): 
	standByCurrentConsumption = 1.25
	activeCurrentConsumption = 135

	averaging = CC_params[0]
	convCycleTime = CC_params[1]

	activeConversionTime = averaging*0.0155
	standbyTime =  convCycleTime- activeConversionTime
	amps = ((activeCurrentConsumption*activeConversionTime) + (standByCurrentConsumption*standbyTime)) / convCycleTime

def OneShot():
	# compute and return power for one shot

In [ ]:
def TMP117data(time, frequency, mode, CC_params, OS_params):
	if mode == "CC": 
		power = ContinuousConversion(CC_params)
		
	elif mode == "OS": 
		power = OneShot(OS_params)
	
	else: # low limit stuff

	return power, time # returns data and time as functions of time


Parameters for TMP117 (to be changed by teams)  
...add more explanation of what each input does later  
add explanation of what possible inputs including default are for each input

In [ ]:
mode = 'OS'
time = 0 # possible inputs: ...
frequency = 0 # possible inputs: ...
conversion = 0 # possible inputs: [000,001,010,011]
CC_params = [convCycle, standByTime, averaging, ]# set to 0 if mode chosen is not CC
OS_params = [] # set to 0 if mode chosen is not OS

power_output = TMP117(time, frequency, mode, CC_params, OS_params) # calling the TMP117 function to output the power 

In [ ]:
![title](/convTimes.png)